# Assignment 2:  Study Helper with Structured Tools

---


## Application with Tools and Function Calling Capabilities

---

## Overview

This assignment enhances the Study Helper agent from Assignment 1 by implementing:
- Structured tool definitions using JSON Schema
- Native LLM function calling capabilities
- Improved error handling and validation
- Multi-parameter tools with different data types

### Key Improvements from Assignment 1:
1. **Structured Tools**: Moved from manual JSON parsing to native function calling
2. **Better Validation**: LLM validates inputs before execution
3. **Industry Standards**: Using JSON Schema for tool definitions
4. **Enhanced Robustness**: Proper error handling and type checking

---

## Part 1: Setup and Environment Configuration

In [1]:
# Step 1: Install required library
!pip install litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 24.5 MB/s eta 0:00:00


In [2]:
# Step 2: Import necessary libraries
import os
import json
from typing import List, Dict, Optional
from litellm import completion

print("Libraries imported successfully")

Libraries imported successfully


In [ ]:
# Step 3: Configure API key
# For Groq API (keys starting with 'gsk_')
os.environ["GROQ_API_KEY"] = "Enter Your APi"

# For OpenAI API (keys starting with 'sk-')
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

print("API key configured")

API key configured


---
## Part 2: Tool Function Implementations

These are the actual Python functions that will be called by the agent.

In [4]:
# Global storage for study notes
study_notes = {}

def generate_quiz(topic: str, num_questions: int = 5, difficulty: str = "medium") -> dict:
    """
    Generate a quiz on a given topic.

    Args:
        topic: The subject for the quiz
        num_questions: Number of questions to generate
        difficulty: Difficulty level (easy, medium, hard)

    Returns:
        Dictionary containing quiz information
    """
    try:
        print(f"Generating {num_questions} {difficulty} questions about {topic}...")

        # Create prompt for quiz generation
        quiz_prompt = [
            {
                "role": "system",
                "content": "You are an expert educational quiz creator."
            },
            {
                "role": "user",
                "content": f"""Create {num_questions} multiple-choice quiz questions about {topic} at {difficulty} difficulty level.

For each question provide:
1. A clear question
2. Four answer options (A, B, C, D)
3. The correct answer letter
4. A brief explanation

Format each question:

Q1: [Question text]
A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]
Correct: [Letter]
Explanation: [Brief explanation]"""
            }
        ]

        # Determine model based on API key
        if os.environ.get("GROQ_API_KEY"):
            model = "groq/llama-3.3-70b-versatile"
        else:
            model = "openai/gpt-4o"

        response = completion(
            model=model,
            messages=quiz_prompt,
            max_tokens=1024
        )

        quiz_content = response.choices[0].message.content

        return {
            "status": "success",
            "topic": topic,
            "num_questions": num_questions,
            "difficulty": difficulty,
            "quiz_content": quiz_content
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("generate_quiz function defined")

generate_quiz function defined


In [5]:
def summarize_text(text: str, max_length: int = 100, include_keywords: bool = True) -> dict:
    """
    Summarize provided text with optional keyword extraction.

    Args:
        text: The text to summarize
        max_length: Target summary length in words
        include_keywords: Whether to extract key terms

    Returns:
        Dictionary containing summary and optional keywords
    """
    try:
        print(f"Summarizing text (target length: {max_length} words, keywords: {include_keywords})...")

        keyword_instruction = "Also extract 5-7 key terms or concepts." if include_keywords else ""

        summary_prompt = [
            {
                "role": "system",
                "content": "You are an expert at creating concise summaries."
            },
            {
                "role": "user",
                "content": f"""Summarize the following text in approximately {max_length} words. {keyword_instruction}

Text:
{text}

Provide:
1. Summary
2. Key points (3-5 bullet points)
{"3. Keywords" if include_keywords else ""}"""
            }
        ]

        if os.environ.get("GROQ_API_KEY"):
            model = "groq/llama-3.3-70b-versatile"
        else:
            model = "openai/gpt-4o"

        response = completion(
            model=model,
            messages=summary_prompt,
            max_tokens=512
        )

        summary_content = response.choices[0].message.content

        return {
            "status": "success",
            "original_length": len(text.split()),
            "target_length": max_length,
            "summary": summary_content,
            "keywords_included": include_keywords
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("summarize_text function defined")

summarize_text function defined


In [6]:
def explain_concept(concept: str, difficulty: str = "simple", include_examples: bool = True) -> dict:
    """
    Explain a concept at different difficulty levels.

    Args:
        concept: The concept to explain
        difficulty: Level of explanation (simple, medium, detailed)
        include_examples: Whether to include practical examples

    Returns:
        Dictionary containing explanation
    """
    try:
        print(f"Explaining '{concept}' at {difficulty} level (examples: {include_examples})...")

        difficulty_instructions = {
            "simple": "Explain in simple terms suitable for beginners. Use analogies.",
            "medium": "Provide a moderate explanation with some technical details.",
            "detailed": "Give a comprehensive explanation with technical depth."
        }

        instruction = difficulty_instructions.get(difficulty, difficulty_instructions["simple"])
        examples_instruction = "Include 2-3 practical examples." if include_examples else "Focus on theory without examples."

        explain_prompt = [
            {
                "role": "system",
                "content": "You are an expert educator."
            },
            {
                "role": "user",
                "content": f"{instruction}\n{examples_instruction}\n\nConcept: {concept}\n\nProvide a clear explanation with key takeaways."
            }
        ]

        if os.environ.get("GROQ_API_KEY"):
            model = "groq/llama-3.3-70b-versatile"
        else:
            model = "openai/gpt-4o"

        response = completion(
            model=model,
            messages=explain_prompt,
            max_tokens=768
        )

        explanation = response.choices[0].message.content

        return {
            "status": "success",
            "concept": concept,
            "difficulty": difficulty,
            "examples_included": include_examples,
            "explanation": explanation
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("explain_concept function defined")

explain_concept function defined


In [7]:
def analyze_text(text: str, language: str = "en", check_grammar: bool = False, word_count: bool = True) -> dict:
    """
    Analyze text for various metrics and properties.
    NEW MULTI-PARAMETER TOOL for Assignment 2.

    Args:
        text: The text to analyze
        language: Language code (en, es, fr, ur)
        check_grammar: Whether to check for grammatical errors
        word_count: Whether to include detailed word statistics

    Returns:
        Dictionary containing analysis results
    """
    try:
        print(f"Analyzing text (language: {language}, grammar: {check_grammar}, stats: {word_count})...")

        # Basic statistics
        words = text.split()
        chars = len(text)
        sentences = text.count('.') + text.count('!') + text.count('?')

        result = {
            "status": "success",
            "language": language,
            "basic_stats": {
                "character_count": chars,
                "word_count": len(words),
                "sentence_count": sentences
            }
        }

        # Detailed word analysis if requested
        if word_count:
            avg_word_length = sum(len(w) for w in words) / len(words) if words else 0
            result["detailed_stats"] = {
                "average_word_length": round(avg_word_length, 2),
                "longest_word": max(words, key=len) if words else None,
                "shortest_word": min(words, key=len) if words else None
            }

        # Grammar check using LLM if requested
        if check_grammar:
            grammar_prompt = [
                {
                    "role": "system",
                    "content": f"You are a grammar expert for {language} language."
                },
                {
                    "role": "user",
                    "content": f"Check this text for grammatical errors and provide brief feedback:\n\n{text}"
                }
            ]

            if os.environ.get("GROQ_API_KEY"):
                model = "groq/llama-3.3-70b-versatile"
            else:
                model = "openai/gpt-4o"

            response = completion(
                model=model,
                messages=grammar_prompt,
                max_tokens=256
            )

            result["grammar_check"] = response.choices[0].message.content

        return result
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("analyze_text function defined (NEW multi-parameter tool)")

analyze_text function defined (NEW multi-parameter tool)


In [8]:
def save_notes(title: str, content: str) -> dict:
    """
    Save study notes for later retrieval.

    Args:
        title: Title for the notes
        content: Content to save

    Returns:
        Dictionary with save status
    """
    try:
        study_notes[title] = content
        return {
            "status": "success",
            "message": f"Notes '{title}' saved successfully",
            "total_notes": len(study_notes)
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("save_notes function defined")

save_notes function defined


In [9]:
def retrieve_notes(title: Optional[str] = None) -> dict:
    """
    Retrieve saved study notes.

    Args:
        title: Specific note title (optional)

    Returns:
        Dictionary with notes or list of titles
    """
    try:
        if title:
            if title in study_notes:
                return {
                    "status": "success",
                    "title": title,
                    "content": study_notes[title]
                }
            else:
                return {
                    "status": "error",
                    "message": f"Notes '{title}' not found"
                }
        else:
            return {
                "status": "success",
                "available_notes": list(study_notes.keys()),
                "total": len(study_notes)
            }
    except Exception as e:
        return {"status": "error", "message": str(e)}

print("retrieve_notes function defined")

retrieve_notes function defined


---
## Part 3: Structured Tool Definitions (JSON Schema)

This is the key improvement from Assignment 1. We define tools using JSON Schema format that LLMs understand natively.

In [10]:
# Map tool names to actual functions
tool_functions = {
    "generate_quiz": generate_quiz,
    "summarize_text": summarize_text,
    "explain_concept": explain_concept,
    "analyze_text": analyze_text,
    "save_notes": save_notes,
    "retrieve_notes": retrieve_notes
}

print("Tool function mapping created")

Tool function mapping created


In [11]:
# Define structured tool schemas for LLM
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_quiz",
            "description": "Generate a multiple-choice quiz on any topic with customizable difficulty and number of questions.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The subject or topic for the quiz"
                    },
                    "num_questions": {
                        "type": "integer",
                        "description": "Number of questions to generate",
                        "default": 5
                    },
                    "difficulty": {
                        "type": "string",
                        "enum": ["easy", "medium", "hard"],
                        "description": "Difficulty level of the quiz",
                        "default": "medium"
                    }
                },
                "required": ["topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_text",
            "description": "Summarize provided text with optional keyword extraction.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to summarize"
                    },
                    "max_length": {
                        "type": "integer",
                        "description": "Target summary length in words",
                        "default": 100
                    },
                    "include_keywords": {
                        "type": "boolean",
                        "description": "Whether to extract key terms",
                        "default": True
                    }
                },
                "required": ["text"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "explain_concept",
            "description": "Explain a concept at different difficulty levels with optional examples.",
            "parameters": {
                "type": "object",
                "properties": {
                    "concept": {
                        "type": "string",
                        "description": "The concept to explain"
                    },
                    "difficulty": {
                        "type": "string",
                        "enum": ["simple", "medium", "detailed"],
                        "description": "Level of explanation detail",
                        "default": "simple"
                    },
                    "include_examples": {
                        "type": "boolean",
                        "description": "Whether to include practical examples",
                        "default": True
                    }
                },
                "required": ["concept"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_text",
            "description": "Analyze text for various metrics including word count, grammar, and language-specific features. NEW multi-parameter tool.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to analyze"
                    },
                    "language": {
                        "type": "string",
                        "enum": ["en", "es", "fr", "ur"],
                        "description": "Language code for analysis",
                        "default": "en"
                    },
                    "check_grammar": {
                        "type": "boolean",
                        "description": "Whether to check for grammatical errors",
                        "default": False
                    },
                    "word_count": {
                        "type": "boolean",
                        "description": "Whether to include detailed word statistics",
                        "default": True
                    }
                },
                "required": ["text"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_notes",
            "description": "Save study notes for later retrieval.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Title for the notes"
                    },
                    "content": {
                        "type": "string",
                        "description": "Content to save"
                    }
                },
                "required": ["title", "content"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_notes",
            "description": "Retrieve saved study notes. If no title provided, lists all available notes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Specific note title to retrieve (optional)"
                    }
                },
                "required": []
            }
        }
    }
]

print(f"Defined {len(tools)} structured tools using JSON Schema")

Defined 6 structured tools using JSON Schema


---
## Part 4: Agent System Rules

In [12]:
# Agent system rules (simplified with native function calling)
agent_rules = [{
    "role": "system",
    "content": """
You are a Study Helper AI agent that assists students with learning.

You have access to several tools for helping students:
- generate_quiz: Create quizzes on any topic
- summarize_text: Summarize long texts
- explain_concept: Explain concepts at different levels
- analyze_text: Analyze text for grammar, statistics, and language features
- save_notes: Save study materials
- retrieve_notes: Access saved materials

Always choose the most appropriate tool for the user's request.
Be helpful, encouraging, and educational in your responses.
"""
}]

print("Agent system rules defined")

Agent system rules defined


---
## Part 5: Agent Loop with Native Function Calling

This is the major improvement from Assignment 1. We now use the LLM's native function calling instead of manual JSON parsing.

In [13]:
def run_enhanced_agent(max_iterations: int = 10):
    """
    Enhanced agent loop using native LLM function calling.

    Key improvements from Assignment 1:
    1. Uses 'tools' parameter in completion call
    2. No manual JSON parsing required
    3. Better error handling with validation
    4. Native tool_calls extraction

    Args:
        max_iterations: Maximum number of interactions
    """
    # Initialize memory
    memory = agent_rules.copy()
    iterations = 0

    print("\n" + "="*60)
    print("Study Helper Agent - Enhanced with Structured Tools")
    print("="*60)
    print("\nAvailable capabilities:")
    print("  - Generate quizzes with custom difficulty")
    print("  - Summarize text with keyword extraction")
    print("  - Explain concepts at different levels")
    print("  - Analyze text (NEW: grammar, stats, multi-language)")
    print("  - Save and retrieve study notes")
    print("\nType 'quit' or 'exit' to end session.")
    print("="*60 + "\n")

    while iterations < max_iterations:
        # Get user input
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("\nSession ended. Thank you for using Study Helper.")
            break

        if not user_input:
            print("Please enter a message.")
            continue

        # Add user message to memory
        memory.append({"role": "user", "content": user_input})

        try:
            # Determine model
            if os.environ.get("GROQ_API_KEY"):
                model = "groq/llama-3.3-70b-versatile"
            else:
                model = "openai/gpt-4o"

            print("\nAgent: Processing...")

            # Call LLM with structured tools (KEY IMPROVEMENT)
            response = completion(
                model=model,
                messages=memory,
                tools=tools,  # Pass structured tool definitions
                max_tokens=1024
            )

            message = response.choices[0].message

            # Check if LLM wants to use a tool
            if hasattr(message, 'tool_calls') and message.tool_calls:
                # Extract tool call (no manual parsing needed!)
                tool_call = message.tool_calls[0]
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)

                print(f"\nCalling tool: {tool_name}")
                print(f"Arguments: {json.dumps(tool_args, indent=2)}")

                # Validate tool exists
                if tool_name not in tool_functions:
                    result = {
                        "status": "error",
                        "message": f"Tool '{tool_name}' not found"
                    }
                else:
                    # Execute the tool function
                    try:
                        result = tool_functions[tool_name](**tool_args)
                    except TypeError as e:
                        result = {
                            "status": "error",
                            "message": f"Invalid arguments for {tool_name}: {str(e)}"
                        }
                    except Exception as e:
                        result = {
                            "status": "error",
                            "message": f"Error executing {tool_name}: {str(e)}"
                        }

                # Display result
                print("\n" + "="*60)
                print("Result:")
                print("="*60)

                if isinstance(result, dict):
                    for key, value in result.items():
                        if key != 'status':
                            if isinstance(value, str) and len(value) > 300:
                                print(f"\n{key}:")
                                print(value)
                            else:
                                print(f"{key}: {value}")
                else:
                    print(result)

                print("="*60)

                # Update memory with assistant message and tool result
                memory.append({
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [{
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_name,
                            "arguments": tool_call.function.arguments
                        }
                    }]
                })

                memory.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })

            else:
                # No tool call, just a regular response
                print(f"\nAgent: {message.content}")
                memory.append({"role": "assistant", "content": message.content})

        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try again.")

        iterations += 1

    if iterations >= max_iterations:
        print("\nMaximum iterations reached. Session ended.")

print("Enhanced agent loop defined")

Enhanced agent loop defined


---
## Part 6: Run the Enhanced Agent

In [14]:
# Run the enhanced agent
run_enhanced_agent(max_iterations=10)


Study Helper Agent - Enhanced with Structured Tools

Available capabilities:
  - Generate quizzes with custom difficulty
  - Summarize text with keyword extraction
  - Explain concepts at different levels
  - Analyze text (NEW: grammar, stats, multi-language)
  - Save and retrieve study notes

Type 'quit' or 'exit' to end session.


You: Create a quiz about Python programming with 3 questions

Agent: Processing...

Calling tool: generate_quiz
Arguments: {
  "num_questions": 3,
  "topic": "Python programming"
}
Generating 3 medium questions about Python programming...

Result:
topic: Python programming
num_questions: 3
difficulty: medium

quiz_content:
Q1: What is the purpose of the "self" parameter in a Python class method?
A) To access global variables
B) To access class variables and methods
C) To access local variables only
D) To exit the program
Correct: B
Explanation: The "self" parameter is a reference to the current instance of the class and is used to access variables and meth

---
## Testing Scenarios

### Test 1: Basic Tool Call
```
You: Create a quiz about Python programming with 3 questions
Expected: Uses generate_quiz with appropriate parameters
```

### Test 2: Multi-Parameter Tool (NEW)
```
You: Analyze this text for grammar and check word count: "The quick brown fox jumps over the lazy dog."
Expected: Uses analyze_text with multiple parameters (text, check_grammar=true, word_count=true)
```

### Test 3: Error Handling
```
You: Generate a quiz on an invalid topic with negative questions
Expected: Graceful error handling with helpful message
```

### Test 4: Memory Integration
```
You: Explain recursion in simple terms
Agent: [Explains recursion]
You: Now create a quiz about what you just explained
Expected: Agent remembers previous explanation and creates relevant quiz
```

### Test 5: Multiple Tools in Sequence
```
You: Create a quiz about Machine Learning
Agent: [Creates quiz]
You: Save this quiz as "ML Basics"
Agent: [Saves the quiz]
You: Show me all my saved notes
Agent: [Lists saved notes including "ML Basics"]
```

---

## Key Improvements from Assignment 1

### 1. Structured Tool Definitions
**Before (Assignment 1):**
```python
# Manual JSON in prompt
"""Respond with:
{
  "tool_name": "generate_quiz",
  "args": {"topic": "...", "num_questions": 5}
}
"""
```

**After (Assignment 2):**
```python
# JSON Schema with validation
{
  "type": "function",
  "function": {
    "name": "generate_quiz",
    "parameters": {
      "type": "object",
      "properties": {...},
      "required": ["topic"]
    }
  }
}
```

### 2. Native Function Calling
**Before:**
```python
# Manual parsing
response = generate_response(messages)
action = parse_action(response)  # Custom JSON extraction
tool_name = action["tool_name"]
```

**After:**
```python
# Native extraction
response = completion(messages=messages, tools=tools)
tool_call = response.choices[0].message.tool_calls[0]
tool_name = tool_call.function.name
```

### 3. Better Error Handling
- Validation of tool existence
- Type checking for arguments
- Graceful failure messages
- Try-except blocks at multiple levels

### 4. Multi-Parameter Tool
The new `analyze_text` function demonstrates:
- String parameters (text, language)
- Boolean parameters (check_grammar, word_count)
- Enum values (language choices)
- Optional parameters with defaults

---

## Benefits of Structured Approach

### 1. Validation
The LLM validates parameter types before calling functions, reducing errors.

### 2. Clarity
JSON Schema provides clear documentation of what each tool does and expects.

### 3. Interoperability
Tools can be shared across different applications and systems.

### 4. Native Support
Modern LLMs (GPT-4, Claude, Llama) support this format natively.

### 5. Maintainability
Easier to add, modify, or remove tools without changing prompts.

### 6. Industry Standard
Follows established patterns used in production systems.

---